# Tutorial Plots

## Imports

In [ ]:
# Make sure that the package can be imported
%matplotlib inline
%config InlineBackend.figure_formats = ['svg']

import sys
from os.path import abspath, join
repo_path = abspath('').split('QuREBB')[0] + 'QuREBB'
sys.path.append(repo_path)
from importlib import reload  # Python 3.4+

import numpy as np
import qutip as qt
import scipy.constants as cst
import matplotlib.pyplot as plt
from datetime import datetime
import xarray as xr
import glob
from lib.protocol import ProtocolSweep

import h5py as h5
save_path=join(repo_path, 'tutorial_simulations/simulation_datasets/')
figure_path=join(repo_path, 'tutorial_simulations/simulation_plots/')



import time
from copy import deepcopy

qt.settings.auto_tidyup = False


## Formatting settings

In [ ]:
import matplotlib as mpl
mpl.rcParams.update(mpl.rcParamsDefault)
params = {
    "xtick.direction": "in",
    "ytick.direction": "in",
    "xtick.top": True,
    "ytick.right": True,
    'font.size': 10,
    'legend.title_fontsize' :'x-small',
}

mpl.rcParams.update(params)

default_colors_light = ['#BF3636', '#6DABDA','#9E79C9','#DB920F', '#13AE74']
default_colors_dark = ['#732020', '#466E8C', '#624B7D', '#8F600A', '#0B6141']

default_shades_blue = ['#6DABDA', '#5B8EB5', '#466E8C', '#1D2E3B', '#324E63']
default_shades_red = ['#ED4242', '#BF3636', '#732020', '#4A1515', '#9C2C2C']

def plot_settings(color_idx=0):
    return {
        'color': default_colors_light[color_idx],
        'marker': '.',
        'markeredgecolor': default_colors_dark[color_idx],
        'linestyle': '--',
    }

sweep_plot_settings={
    'marker': '.',
    'linestyle': '--',
}

dub_col = (2 * 246 + 18) / 72
fig_height = 0.6 * dub_col

## Helper Functions

In [ ]:
import matplotlib.ticker as mticker
class AdditionalTickLocator(mticker.Locator):
    '''This locator chains whatever locator given to it, and then add addition custom ticks to the result'''
    def __init__(self, chain: mticker.Locator, ticks) -> None:
        super().__init__()
        assert chain is not None
        self._chain = chain
        self._additional_ticks = np.asarray(list(ticks))

    def _add_locs(self, locs):
        locs = np.unique(np.concatenate([
            np.asarray(locs),
            self._additional_ticks
        ]))
        return locs

    def tick_values(self, vmin, vmax):
        locs = self._chain.tick_values(vmin, vmax)
        return self._add_locs(locs)

    def __call__(self):
        # this will call into chain's own tick_values,
        # so we also add ours here
        locs = self._chain.__call__()
        return self._add_locs(locs)

    def nonsingular(self, v0, v1):
        return self._chain.nonsingular(v0, v1)
    def set_params(self, **kwargs):
        return self._chain.set_params(**kwargs)
    def view_limits(self, vmin, vmax):
        return self._chain.view_limits(vmin, vmax)


class AdditionalTickFormatter(mticker.Formatter):
    '''This formatter chains whatever formatter given to it, and
    then does special formatting for those passed in custom ticks'''
    def __init__(self, chain: mticker.Formatter, ticks) -> None:
        super().__init__()
        assert chain is not None
        self._chain = chain
        self._additional_ticks = ticks

    def __call__(self, x, pos=None):
        if x in self._additional_ticks:
            return self._additional_ticks[x]
        res = self._chain.__call__(x, pos)
        return res

    def format_data_short(self, value):
        if value in self._additional_ticks:
            return self.__call__(value)
        return self._chain.format_data_short(value)

    def get_offset(self):
        return self._chain.get_offset()
    
    def _set_locator(self, locator):
        self._chain._set_locator(locator)

    def set_locs(self, locs):
        self._chain.set_locs(locs)
        
def axis_add_custom_ticks(axis, ticks):
    locator = axis.get_major_locator()
    formatter = axis.get_major_formatter()
    axis.set_major_locator(AdditionalTickLocator(locator, ticks.keys()))
    axis.set_major_formatter(AdditionalTickFormatter(formatter, ticks))

def find_latest_dataset(label, save_folder):
    files = glob.glob( save_path + f"*{label}*")
    dataset_names = [split(x)[-1] for x in files]
    ds_timestamps = [datetime.strptime(x[:15], '%Y%m%d-%H%M%S') for x in dataset_names]

    sorted_idx = np.argsort(ds_timestamps)

    return join(save_folder, dataset_names[sorted_idx[-1]])

# Basic Params

In [ ]:
file_path = find_latest_dataset("ProtocolA_fidelity_rate", save_path)
print(file_path[len(save_path):])
dsA = xr.load_dataset(file_path)

In [ ]:
file_path = find_latest_dataset("ProtocolB_fidelity_rate", save_path)
print(file_path[len(save_path):])
dsB = xr.load_dataset(file_path)

In [ ]:
file_path = find_latest_dataset("ProtocolC_fidelity_rate", save_path)
print(file_path[len(save_path):])
dsC = xr.load_dataset(file_path)

In [ ]:
file_path = find_latest_dataset("ProtocolC_WCS_fidelity_rate", save_path)
print(file_path[len(save_path):])
dsC_wcs = xr.load_dataset(file_path)

In [ ]:
fig, ax = plt.subplots(figsize = (0.4*dub_col, 0.3*dub_col))

ax.hlines(0.5, 1e-15,0.1, 'k', linestyle = '--', linewidth = 0.8)

(1-dsA.fidelity).plot(ax=ax, label='Protocol A', **plot_settings(0))
(1-dsB.fidelity).plot(ax=ax, label='Protocol B', **plot_settings(4))
(1-dsC.fidelity).plot(ax=ax, label='Protocol C', **plot_settings(1))
(1-dsC_wcs.fidelity).plot(ax=ax, label='Protocol C \n with WCS', **plot_settings(2), zorder=-1)


fig.legend(loc=2, bbox_to_anchor = (0.11, 0.86),  fontsize=8)
ax.set_xlabel(r"Success probability per attempt $(\eta)$")
ax.set_ylabel("Infidelity")
ax.set_ylim(2e-2,0.6)
ax.set_xlim(1e-10,2e-2)
ax.set_xscale('log')
ax.set_yscale('log')

locmin = mpl.ticker.LogLocator(base=10.0, subs=np.arange(2, 10) * .1,
                                      numticks=100)
ax.xaxis.set_minor_locator(locmin)
ax.xaxis.set_minor_formatter(mpl.ticker.NullFormatter())

axis_add_custom_ticks(ax.yaxis, {
    0.5: '0.5'
})

fig.savefig(f'{figure_path}comparison.pdf', bbox_inches="tight", transparent=True)
fig

# Cooperativity sweep

## sweeping kappa

In [ ]:
# Load data

name = "CoopSweepKappa"

coops = [1, 10, 50, 500]

dsk = {}

for c in coops:
    fname = find_latest_dataset(name+f"_C{c}_fidelity_rate.", save_path)
    print(fname[len(save_path):])
    d = xr.load_dataset(fname)
    
    dsk.update({str(c): d})

In [ ]:
# Plot

fig, ax = plt.subplots(figsize=(0.5*dub_col, 0.25*dub_col))

i=0
for key, el in dsk.items():
    (1-el.fidelity).plot(ax=ax, label = f"C={key}", color=default_shades_blue[i], 
                         **sweep_plot_settings)
    i+=1

ax.set_xscale('log')
ax.set_yscale('log')

ax.vlines(1e-4, 0, 0.6, color = default_shades_blue[4], linestyle='--', linewidth = 0.8, zorder=-1)
ax.vlines(1e-5, 0, 0.6, color = default_shades_blue[0], linestyle='--', linewidth = 0.8, zorder=-1)

ax.set_ylim(5e-3,0.6)
ax.set_xlim(1e-6,1e-3)
ax.set_xlabel(r"Success probability per attempt $(\eta)$")
ax.set_ylabel("Infidelity")

fig.legend(loc=1, bbox_to_anchor=(0.91, 0.48), fontsize=7)


axis_add_custom_ticks(ax.yaxis, {
    0.5: '0.5'
})


fig.savefig(f'{figure_path}coop_kappa.pdf', bbox_inches="tight", transparent=True)
fig

## Sweeping g

In [ ]:
# Load data

name = "CoopSweepG"

coops = [1, 10, 50, 500]

dsg = {}

for c in coops:
    fname = find_latest_dataset(name+f"_C{c}_fidelity_rate", save_path)
    print(fname[len(save_path):])
    d = xr.load_dataset(fname)
    
    dsg.update({str(c): d})

In [ ]:
# Plot

fig, ax = plt.subplots(figsize=(0.5*dub_col, 0.25*dub_col))

sweep_plot_settings.update({'marker':'.'})

i=0
for key, el in dsg.items():
    (1-el.fidelity).plot(ax=ax, label = f"C={key}", color = default_shades_red[i], **sweep_plot_settings)
    i +=1

ax.set_xscale('log')
ax.set_yscale('log')

ax.set_ylim(5e-3,0.6)
ax.set_xlim(1e-6,1e-3)
ax.set_xlabel(r"Success probability per attempt $(\eta)$")
ax.set_ylabel("Infidelity")

fig.legend(loc=1, bbox_to_anchor=(0.91, 0.48), fontsize=7)

ax.vlines(1e-4, 0, 0.6, color = default_shades_red[4], linestyle='--', linewidth = 0.8, zorder=-1)
ax.vlines(1e-5, 0, 0.6, color = default_shades_red[0], linestyle='--', linewidth = 0.8, zorder=-1)


axis_add_custom_ticks(ax.yaxis, {
    0.5: '0.5'
})


fig.savefig(f'{figure_path}coop_g.pdf', bbox_inches="tight", transparent=True)
fig

## Best Cooperativity for Rate

In [ ]:
# Laod data

coops = [1, 10, 50, 100, 200, 500, 1000]

lowrate = 1e-5
hirate=1e-4

max_lowrate_kappa = []
max_lowrate_g = []

max_hirate_kappa = []
max_hirate_g = []

dss = []

for c in coops:
    fname = find_latest_dataset("CoopSweepG"+f"_C{c}_fidelity_rate", save_path)
    print(fname[len(save_path):])
    d = xr.load_dataset(fname)

    max_lowrate_g.append( float(d.sel(rate=lowrate, method='nearest').fidelity.values) )
    max_hirate_g.append(  float(d.sel(rate=hirate, method='nearest').fidelity.values) )

    fname = find_latest_dataset("CoopSweepKappa"+f"_C{c}_fidelity_rate", save_path)
    print(fname[len(save_path):])
    d = xr.load_dataset(fname)

    dss.append(d)
    max_lowrate_kappa.append( float(d.sel(rate=lowrate, method='nearest').fidelity.values) )
    max_hirate_kappa.append(  float(d.sel(rate=hirate, method='nearest').fidelity.values) )

In [ ]:
# Plot

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(0.5*dub_col, 0.3*dub_col), sharey=True)
plt.subplots_adjust(wspace = 0.1)

ax1.plot(coops, 1-np.array(max_lowrate_kappa), label = r"$\eta=10^{-5}$", 
        markeredgecolor=default_shades_blue[1], color=default_shades_blue[0], marker='o')
ax1.plot(coops, 1-np.array(max_hirate_kappa), label = r"$\eta=10^{-4}$", 
        markeredgecolor=default_shades_blue[3], color=default_shades_blue[4], marker='o')

ax2.plot(coops, 1-np.array(max_lowrate_g), label = r"$\eta =10^{-5}$", 
        markeredgecolor=default_shades_red[1], color=default_shades_red[0], marker='v')
ax2.plot(coops, 1-np.array(max_hirate_g), label = r"$\eta=10^{-4}$", 
        markeredgecolor=default_shades_red[3],color=default_shades_red[4], marker='v')



ax1.set_yscale('log')
ax1.set_ylabel('Infidelity')

ax1.set_xlabel("Cooperativity")
ax2.set_xlabel("Cooperativity")

ax1.legend(loc=1, bbox_to_anchor=(1.03, 0.40), fontsize=8, title=r"Varying $\kappa$")
ax2.legend(loc=1, bbox_to_anchor=(1.03, 0.40), fontsize=8, title=r"Varying $g$")


fig.savefig(f'{figure_path}coop_max.pdf', bbox_inches="tight", transparent=True)
fig